### Import libraries 

In [1]:
import numpy as np
import pandas as pd

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, FactorRange, LabelSet, PrintfTickFormatter, SingleIntervalTicker, Title 

output_notebook()

Loading BokehJS ...

### Goal 

<img src="../images/original/fig_0603.png">

#### Colors 

In [2]:
GRAY1, GRAY2, GRAY3 = '#231F20', '#414040', '#555655'
GRAY4, GRAY5, GRAY6 = '#646369', '#76787B', '#828282'
GRAY7, GRAY8, GRAY9 = '#929497', '#A6A6A5', '#BFBEBE'
BLUE1, BLUE2, BLUE3, BLUE4 = '#174A7E', '#4A81BF', '#94B2D7', '#94AFC5'
RED1, RED2, RED3 = '#800000', '#C3514E', '#E6BAB7', 
GREEN1, GREEN2 = '#0C8040', '#9ABB59'
ORANGE1 = '#F79747'

#### Font 

In [3]:
FONT = 'Arial'

#### Preprocessing 

In [4]:
dataset = pd.read_csv('../data/data_fig_0603.csv')
dataset.shape

(11, 5)

In [5]:
dataset.head()

,Year,Quarter,Miss,Meet,Exceed
0,2013,Q1,5,53,42
1,2013,Q2,3,62,35
2,2013,Q3,6,68,26
3,2013,Q4,5,70,25
4,2014,Q1,6,75,19


In [6]:
dataset['Year'] = dataset['Year'].astype(str)
dataset['Year Quarter'] = list(zip(dataset['Year'], dataset['Quarter']))
dataset = dataset.drop(columns=['Year', 'Quarter'])

In [7]:
dataset.head()

,Miss,Meet,Exceed,Year Quarter
0,5,53,42,"(2013, Q1)"
1,3,62,35,"(2013, Q2)"
2,6,68,26,"(2013, Q3)"
3,5,70,25,"(2013, Q4)"
4,6,75,19,"(2014, Q1)"


In [8]:
dataset_vbar = dataset.set_index('Year Quarter').T.rename_axis('Type').reset_index()
dataset_vbar.columns.name = None

In [9]:
dataset_vbar

,Type,"(2013, Q1)","(2013, Q2)","(2013, Q3)","(2013, Q4)","(2014, Q1)","(2014, Q2)","(2014, Q3)","(2014, Q4)","(2015, Q1)","(2015, Q2)","(2015, Q3)"
0,Miss,5,3,6,5,6,6,12,15,20,33,42
1,Meet,53,62,68,70,75,59,61,62,68,62,47
2,Exceed,42,35,26,25,19,35,27,23,12,5,11


In [10]:
# Create datasource for labels
datalabel = dataset.loc[6:10].copy()
datalabel['Formatted Miss'] = datalabel['Miss'].astype(int).astype(str) + '%'
datalabel['Location'] = [12, 15, 18, 28, 35]

In [11]:
datalabel

,Miss,Meet,Exceed,Year Quarter,Formatted Miss,Location
6,12,61,27,"(2014, Q3)",12%,12
7,15,62,23,"(2014, Q4)",15%,15
8,20,68,12,"(2015, Q1)",20%,18
9,33,62,5,"(2015, Q2)",33%,28
10,42,47,11,"(2015, Q3)",42%,35


#### Plot 

In [12]:
dataset_vbar

,Type,"(2013, Q1)","(2013, Q2)","(2013, Q3)","(2013, Q4)","(2014, Q1)","(2014, Q2)","(2014, Q3)","(2014, Q4)","(2015, Q1)","(2015, Q2)","(2015, Q3)"
0,Miss,5,3,6,5,6,6,12,15,20,33,42
1,Meet,53,62,68,70,75,59,61,62,68,62,47
2,Exceed,42,35,26,25,19,35,27,23,12,5,11


In [13]:
# Set the source of the plot
source = ColumnDataSource(dataset)
source_label = ColumnDataSource(datalabel)


customer = dataset_vbar['Type'].to_list() # List to stacked bar
segments = dataset['Year Quarter'].to_list() # List to set x range


# Set color by category to the stacked bar
colors = [RED1, GRAY9, GRAY4]


# Create the figure
p = figure(x_range=FactorRange(*segments),
           y_range=(0, 130),
           plot_height=550,
           plot_width=900, 
           title='Goal attainment over time',
           toolbar_location='above')


# Add bars to the figure
p.vbar_stack(customer, 
             x='Year Quarter',
             color=colors,
             width=0.7, 
             legend_label=customer,
             line_color='WHITE',
             line_width=1,
             source=source)


# Add labels to the bars
p.add_layout(LabelSet(x='Year Quarter', 
                      y='Location', 
                      x_offset=0, 
                      y_offset=-5, 
                      text='Formatted Miss',                      
                      text_align='center',
                      text_baseline='top', 
                      text_color='white',
                      text_font=FONT,
                      text_font_size='14pt',
                      source=source_label))


p.text(x=7, 
       y=110,
       x_offset=47,
       y_offset=0,
       text=['As of Q3 2015,'], 
       text_color=GRAY4,
       text_font=FONT,
       text_font_size='12pt')

p.text(x=7, 
       y=105,
       x_offset=47,
       y_offset=7,
       text=['\t' * 25 + 'more than 1/3 of\n' + '\t'* 8 + 'projects are missing goals'], 
       text_color=RED1,
       text_font=FONT,
       text_font_size='12pt',
       text_font_style='bold')

# Add footing legend
p.add_layout(Title(text=" "), place="below")

p.add_layout(Title(text='Data source: XYZ Dashboard; the total number of projects has increased over time from 230 in early 2013 to nearly 270 in Q3 2015.', 
                   align="left",
                   offset=-60,
                   text_color=GRAY6,
                   text_font=FONT,
                   text_font_size='11pt',
                   text_font_style='normal'), 
             "below")


# Elements attributes

# Modify title attributes
p.title.offset = -80
p.title.text_color = GRAY4
p.title.text_font = FONT
p.title.text_font_size = '18pt'
p.title.text_font_style = 'normal'


# Modify x axis attributes
p.xaxis.axis_line_color = GRAY9
p.xaxis.axis_line_width = 0.5
p.xaxis.axis_label = None 
p.xaxis.major_label_standoff = 5
p.xaxis.major_label_text_color = GRAY2
p.xaxis.major_label_text_font = FONT
p.xaxis.major_label_text_font_size = '14pt'
p.xaxis.major_tick_line_color = None
p.xaxis.minor_tick_line_color = None
p.xgrid.grid_line_color = None
p.x_range.group_padding = 0
p.below[0].group_text_color = GRAY2
p.below[0].group_text_font_size = '14pt'
p.below[0].group_text_font_style = 'normal'


# Modify y axis attributes
p.yaxis.bounds=(0, 100)
p.yaxis.formatter = PrintfTickFormatter(format='%0.0f %%')
p.yaxis.axis_line_color = GRAY9
p.yaxis.axis_label = ' ' * 18 + '% of total projects'    
p.yaxis.axis_line_width = 0.5
p.yaxis.axis_label_standoff = 5  
p.yaxis.axis_label_text_color = GRAY4
p.yaxis.axis_label_text_font = FONT
p.yaxis.axis_label_text_font_size = '13pt'
p.yaxis.axis_label_text_font_style = 'normal' 
p.yaxis.major_label_standoff = 5
p.yaxis.major_label_text_color = GRAY4
p.yaxis.major_label_text_font = FONT
p.yaxis.major_label_text_font_size = '11.5pt'
p.yaxis.major_tick_in = 0
p.yaxis.major_tick_out = 3
p.yaxis.major_tick_line_color = None
p.yaxis.minor_tick_line_color = None
p.yaxis.ticker = SingleIntervalTicker(interval=10, num_minor_ticks=10)
p.ygrid.grid_line_color = None


# Modify legends axis attributes
p.legend.border_line_color = None
p.legend.orientation = 'horizontal'
p.legend.location = 'top_left'
p.legend.glyph_height = 12
p.legend.glyph_width = 12
p.legend.label_text_color = GRAY4
p.legend.label_text_baseline  = 'alphabetic'
p.legend.label_text_font_size = '10pt'
p.legend.label_text_font_style = 'bold'
p.legend.label_standoff = 10
p.legend.items.name = None
p.legend.spacing = 10


# Handle backgrounds color
p.background_fill_color = 'white'
p.border_fill_color = 'white'
p.outline_line_color = 'white'


show(p)